In [1]:
import pandas as pd

In [16]:
x = pd.read_csv('./features_reduced_imputed.csv')
x.drop(['Unnamed: 0'], axis = 1, inplace = True)
x.shape

(1700, 92)

# 1. binary column중 unbalance한 Feature 삭제 therehold = 0.01

In [15]:
binary_column_info = {}

# 각 컬럼 확인
for column in x.columns:
    # 유일한 값들 확인
    unique_values = x[column].dropna().unique()
    
        # 0과 1만 있는지 확인
    if set(unique_values) <= {0, 1}:
        count_0 = (x[column] == 0).sum()
        count_1 = (x[column] == 1).sum()
        total_count = count_0 + count_1
        percent_0 = (count_0 / total_count) * 100
        percent_1 = (count_1 / total_count) * 100
        binary_column_info[column] = {
            '0 count': count_0, 
            '1 count': count_1,
            '0 percent': percent_0,
            '1 percent': percent_1
        }

cnt = 0
for column, counts in binary_column_info.items():
    
    if(counts['1 percent'] <= 2):
        cnt +=1
        print(column, counts['0 count'], counts['1 count'], counts['0 percent'], counts['1 percent'])
    '''
    print(f"Column: {column}")
    print(f"  0 count: {counts['0 count']}, 1 count: {counts['1 count']}")
    print(f"  0 percent: {counts['0 percent']:.2f}%, 1 percent: {counts['1 percent']:.2f}%")
    '''
print(cnt)

nr_01 1696 4 99.76470588235294 0.2352941176470588
nr_04 1671 29 98.29411764705883 1.7058823529411766
nr_07 1699 1 99.94117647058823 0.0588235294117647
nr_08 1696 4 99.76470588235294 0.2352941176470588
np_01 1698 2 99.88235294117646 0.1176470588235294
np_04 1697 3 99.82352941176471 0.17647058823529413
np_05 1689 11 99.3529411764706 0.6470588235294118
np_07 1699 1 99.94117647058823 0.0588235294117647
endocr_03 1687 13 99.23529411764706 0.7647058823529412
zab_leg_04 1691 9 99.47058823529412 0.5294117647058824
SVT_POST 1692 8 99.52941176470588 0.4705882352941176
GT_POST 1692 8 99.52941176470588 0.4705882352941176
FIB_G_POST 1685 15 99.11764705882354 0.8823529411764706
n_r_ecg_p_08 1696 4 99.76470588235294 0.2352941176470588
n_r_ecg_p_09 1698 2 99.88235294117646 0.1176470588235294
n_r_ecg_p_10 1698 2 99.88235294117646 0.1176470588235294
n_p_ecg_p_01 1698 2 99.88235294117646 0.1176470588235294
fibr_ter_05 1696 4 99.76470588235294 0.2352941176470588
fibr_ter_07 1694 6 99.6470588235294 0.35294

# 2. VAE Test

In [34]:
import tensorflow as tf
from tensorflow.keras import layers, models, backend as K
import pandas as pd

# Load the data
file_path = './features_reduced_imputed.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the data
data.head()

# Define the encoder
input_dim = data.shape[1]
latent_dim = 2  # You can adjust the dimensionality of the latent space

inputs = layers.Input(shape=(input_dim,))
h = layers.Dense(64, activation='relu')(inputs)
h = layers.Dense(32, activation='relu')(h)
z_mean = layers.Dense(latent_dim)(h)
z_log_var = layers.Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

# Define the decoder
decoder_h = layers.Dense(32, activation='relu')
decoder_h2 = layers.Dense(64, activation='relu')
decoder_mean = layers.Dense(input_dim, activation='sigmoid')

h_decoded = decoder_h(z)
h_decoded = decoder_h2(h_decoded)
outputs = decoder_mean(h_decoded)

# Define the VAE model
vae = models.Model(inputs, outputs)

# Define the loss
reconstruction_loss = tf.keras.losses.mse(inputs, outputs)
reconstruction_loss *= input_dim
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [35]:
import tensorflow as tf
print(tf.__version__)

2.16.1
